In [19]:
import sys
sys.path.append(r"../")

from src.data.data_preprocessing import transform_combined_to_classified

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [83]:
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from datasets import Dataset

In [5]:
tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
batch = tokenizer.encode('Today we have shitty weather', return_tensors='pt')

toxic = model(batch).logits.squeeze().tolist()[0] <= 0
print(toxic)

True


In [21]:
df = pd.read_csv('../data/interim/combined.tsv', sep='\t', header=0)
df = transform_combined_to_classified(df)
df.head()

Processing:   0%|          | 0/1262462 [00:00<?, ?it/s]

,text,toxic
0,alkar flood mental wast would explain high lev...,1
1,your becom disgust,1
2,well spare life,1
3,monkey wake,1
4,order kill,1


In [58]:
max_input_length = 128
all_texts = tokenizer(df['text'].to_list(), return_tensors='pt', padding=True, truncation=True, max_length=max_input_length)

In [80]:
all_texts['input_ids']

tensor([[    0,  9707,   271,  ...,     1,     1,     1],
        [    0, 16625, 44949,  ...,     1,     1,     1],
        [    0,  3056, 10628,  ...,     1,     1,     1],
        ...,
        [    0,   298,  7363,  ...,     1,     1,     1],
        [    0,  4892,   428,  ...,     1,     1,     1],
        [    0, 45955,  2694,  ...,     1,     1,     1]])

In [93]:
outputs = []
for i in tqdm(range(0, len(all_texts['input_ids']))):
    outputs.append(model(all_texts['input_ids'][i].unsqueeze(0)).logits.squeeze().tolist()[0] <= 0)

  0%|          | 0/1258981 [00:00<?, ?it/s]

KeyboardInterrupt: 

[2.718515157699585, 2.257903814315796]